!pip install BeautifulSoup4, lxml, pandas, requests, IPython, googlemaps

In [ ]:
import json, config
from urllib.request import urlopen
from bs4 import BeautifulSoup
import googlemaps

In [ ]:
gmaps = googlemaps.Client(key=config.google_api_key)

In [ ]:
url = "https://en.wikipedia.org/wiki/List_of_current_Major_League_Baseball_stadiums"

In [ ]:
page = urlopen(url)

In [ ]:
soup = BeautifulSoup(page, "lxml")

In [ ]:
first_table = soup.find(
    'table', class_='wikitable sortable plainrowheaders')


In [ ]:
A=[] #IMG
B=[] #Arena
C=[] #Capacity
D=[] #City
E=[] #Surface
F=[] #Team
G=[] #Opened
H=[] #Design
I=[] #Roof Type
J=[] #State

In [ ]:
for row in first_table.findAll('tr'):
    cells=row.findAll(['td','th'])
    if cells[0].find(scope_=lambda x: x != 'col'):
        A.append(cells[0].find('img')['src'].replace("120px", "250px"))
        B.append(cells[1].find(text=True))
        ###this splits the city and state then puts them in their own arrays.
        location = cells[3].find(text=True).split(',')
        D.append(location[0])
        J.append(location[1].lstrip())
        C.append(cells[2].find(text=True).rstrip('\n'))
        E.append(cells[4].find(text=True).rstrip('\n'))
        F.append(cells[5].find(text=True))
        G.append(cells[6].find(text=True).rstrip('\n'))
        H.append(cells[8].find(text=True))
        I.append(cells[9].find(text=True).rstrip('\n'))


In [ ]:
import pandas as pd
df=pd.DataFrame(F,columns=['Team'])
df['Arena']=B
df['City']=D
df['State'] = J
df['Capacity']=C
df['opened']=G
df['surface']=E
df['Roof Type']=I
df['Design']=H
df['image']=A
df

In [ ]:
location = [];
for i in range(len(B)):
    searchTerm = B[i] + ", " + J[i]
    geocode_result = gmaps.geocode(searchTerm)
    data = geocode_result[0]['geometry']['location']
    tempArray = []
    tempArray.append(float(data['lng']))
    tempArray.append(float(data['lat']))
    location.append(tempArray[::-1])

response = geocoder.forward(B[0] + ", " + D[0] + ", " + J[0])
collection = response.json()
data = collection['features'][0]['geometry']['coordinates']


In [ ]:
final_data = []
for i in range(len(A)):
    temp={}
    temp['Image'] = "HTTPS:" + A[i]
    temp['Team'] = F[i]
    temp['Arena'] = B[i]
    temp['Location'] = D[i] + ", " + J[i]
    temp['Capacity'] = C[i]
    temp['opened'] = G[i]
    temp['Surface'] = E[i]
    temp['Roof Type'] = I[i]
    temp['Design'] = H[i]
    temp['Coordinates'] = location[i]
    final_data.append(temp)

In [ ]:
print(len(final_data))

In [ ]:
with open('MLB.json', 'w') as outfile:
    json.dump(final_data, outfile)

In [ ]:
import requests, IPython.display as Disp
url = final_data[6]['Image']
Disp.Image(requests.get(url).content)

In [ ]:
print(url)